# 🧬 Cortex-2: Laboratorio de Evolución de IA

Bienvenido al **Cortex-2 Evolutionary Lab**. Este no es un simple script de entrenamiento; es un entorno de investigación para **descubrir** la arquitectura de IA perfecta.

## 🎯 El Objetivo
En lugar de asumir que "Transformers es todo lo que necesitas", usaremos **Algoritmos Genéticos** para evolucionar una arquitectura híbrida que combine lo mejor de varios mundos:
1.  **Mamba (SSM)**: Memoria infinita y eficiente.
2.  **Attention**: Razonamiento complejo inmediato.
3.  **MoE (Mixture of Experts)**: Especialización neuronal.

## 🔬 Caja Blanca: Visualización Total
En este notebook podrás ver:
- **El ADN de la IA**: Cómo se codifica la arquitectura.
- **Evolución en Tiempo Real**: Gráficos interactivos de la supervivencia del más apto.
- **Espacio Latente 3D**: Navega por el "cerebro" del modelo para ver cómo agrupa conceptos.

---

In [ ]:
# 0. Preparación del Laboratorio
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import copy
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from sklearn.decomposition import PCA
from IPython.display import clear_output, display, HTML

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🚀 Sistema Iniciado en: {device.upper()}")

# Semilla para reproducibilidad (opcional, para experimentos científicos)
torch.manual_seed(42)

## 🧬 Módulo 1: El Genoma (DNA)

**¿Por qué esto?**
La mayoría de los investigadores eligen hiperparámetros (capas, cabezas) "a ojo". Nosotros definimos un **Espacio de Búsqueda** y dejamos que la evolución encuentre la combinación óptima.

**Variantes Interesantes:**
- **Gen `backbone_type`**: Puede ser 'transformer' (clásico), 'mamba' (rápido) o 'hybrid' (lo mejor de ambos).
- **Gen `n_experts`**: Define cuánto se especializa el cerebro (MoE).

In [ ]:
class Genome:
    def __init__(self):
        # Definimos los genes y sus posibles alelos (valores)
        self.genes = {
            'n_layers': [2, 4, 6, 8],           # Profundidad
            'd_model': [128, 256, 384],         # Capacidad de abstracción
            'n_heads': [2, 4, 8],               # Paralelismo de atención
            'backbone': ['transformer', 'hybrid'], # Tipo de cerebro
            'moe_experts': [0, 4, 8],           # 0 = Denso, >0 = MoE
            'learning_rate': [1e-3, 3e-4, 1e-4] # Velocidad de aprendizaje
        }
        self.dna = self.random_dna()
        self.fitness = 0.0

    def random_dna(self):
        return {k: random.choice(v) for k, v in self.genes.items()}

    def mutate(self):
        """ Cambia un gen aleatoriamente """
        gene_to_mutate = random.choice(list(self.genes.keys()))
        self.dna[gene_to_mutate] = random.choice(self.genes[gene_to_mutate])
        return f"🧬 Mutación: {gene_to_mutate} -> {self.dna[gene_to_mutate]}"

    def crossover(self, other_genome):
        """ Mezcla genes con otro individuo """
        child = Genome()
        for gene in self.genes:
            # 50% de probabilidad de heredar de padre o madre
            child.dna[gene] = self.dna[gene] if random.random() > 0.5 else other_genome.dna[gene]
        return child

# Ejemplo de un individuo
adam = Genome()
print("Primer Individuo (Adán):", adam.dna)

## 🧠 Módulo 2: El Cerebro (Arquitectura Modular)

Aquí construimos el modelo basándonos en el ADN. 

**Justificación Técnica:**
- **ByteEmbedding**: Usamos bytes (0-255) en lugar de tokens BPE. *Ventaja*: Vocabulario pequeño, robusto a errores ortográficos, multilingüe nativo. *Desventaja*: Secuencias más largas (por eso necesitamos Mamba).
- **Mamba Block**: Usamos una implementación simplificada en PyTorch para portabilidad. En producción usaríamos kernels CUDA.
- **MoE (Mixture of Experts)**: Permite tener muchos parámetros pero usar pocos por token (Inferencia rápida).

In [ ]:
# --- Bloques Constructivos ---

class MambaBlock(nn.Module):
    """ Bloque de Memoria de Estado (Simplificado) """
    def __init__(self, d_model):
        super().__init__()
        self.in_proj = nn.Linear(d_model, d_model * 2)
        self.out_proj = nn.Linear(d_model, d_model)
        # Simulación de la dinámica de estado
        self.conv = nn.Conv1d(d_model, d_model, kernel_size=3, padding=1, groups=d_model)
        
    def forward(self, x):
        # x: (B, L, D)
        B, L, D = x.shape
        x_and_res = self.in_proj(x)
        x_val, res = x_and_res.chunk(2, dim=-1)
        
        # "Scan" simulado con convolución (en realidad sería recurrente)
        x_val = x_val.transpose(1, 2)
        x_val = self.conv(x_val)
        x_val = x_val.transpose(1, 2)
        
        x_val = F.silu(x_val)
        return self.out_proj(x_val * F.sigmoid(res))

class MoELayer(nn.Module):
    """ Capa de Expertos """
    def __init__(self, d_model, n_experts, top_k=2):
        super().__init__()
        self.experts = nn.ModuleList([
            nn.Sequential(
                nn.Linear(d_model, 4 * d_model), nn.GELU(), 
                nn.Linear(4 * d_model, d_model), nn.Dropout(0.1)
            ) for _ in range(n_experts)
        ])
        self.gate = nn.Linear(d_model, n_experts)
        self.top_k = top_k

    def forward(self, x):
        # Gating simple
        gate_logits = self.gate(x)
        weights, indices = torch.topk(gate_logits, self.top_k, dim=-1)
        weights = F.softmax(weights, dim=-1)
        
        out = torch.zeros_like(x)
        # Iteramos (lento en Python, rápido en CUDA/Triton)
        for i, expert in enumerate(self.experts):
            mask = (indices == i).any(dim=-1)
            if mask.any():
                out[mask] += expert(x[mask])
        return out

class CortexOrganism(nn.Module):
    """ El Organismo que evoluciona """
    def __init__(self, dna):
        super().__init__()
        self.dna = dna
        self.embedding = nn.Embedding(256, dna['d_model']) # Byte-level
        self.layers = nn.ModuleList()
        
        for i in range(dna['n_layers']):
            # Lógica Genética: ¿Qué tipo de capa construimos?
            if dna['backbone'] == 'mamba':
                self.layers.append(MambaBlock(dna['d_model']))
            elif dna['backbone'] == 'hybrid' and i % 2 == 0:
                self.layers.append(MambaBlock(dna['d_model']))
            else:
                # Transformer clásico
                self.layers.append(nn.TransformerEncoderLayer(
                    d_model=dna['d_model'], nhead=dna['n_heads'], 
                    dim_feedforward=4*dna['d_model'], batch_first=True
                ))
        
        # Capa final: ¿MoE o Densa?
        if dna['moe_experts'] > 0:
            self.final_layer = MoELayer(dna['d_model'], dna['moe_experts'])
        else:
            self.final_layer = nn.Linear(dna['d_model'], dna['d_model']) # Identidad/Proyección
            
        self.ln_f = nn.LayerNorm(dna['d_model'])
        self.head = nn.Linear(dna['d_model'], 256)

    def forward(self, idx, targets=None):
        x = self.embedding(idx)
        for layer in self.layers:
            x = layer(x)
        
        if self.dna['moe_experts'] > 0:
            x = self.final_layer(x)
            
        x = self.ln_f(x)
        logits = self.head(x)
        
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))
        return logits, loss

## 🔬 Módulo 3: El Microscopio (Visualización 3D)

Para entender la "Caja Blanca", necesitamos ver qué está aprendiendo el modelo. Usaremos **PCA** para reducir las dimensiones de los embeddings y **Plotly** para navegar en 3D.

In [ ]:
def visualize_brain_3d(model):
    """ Visualiza el espacio de conceptos del modelo """
    # Extraemos los embeddings (conceptos que el modelo entiende)
    embeddings = model.embedding.weight.detach().cpu().numpy()
    
    # Reducimos a 3 dimensiones
    pca = PCA(n_components=3)
    components = pca.fit_transform(embeddings)
    
    # Mapeamos bytes a caracteres legibles para la etiqueta
    labels = [chr(i) if 32 <= i <= 126 else f"<{i}>" for i in range(256)]
    
    df = pd.DataFrame(components, columns=['x', 'y', 'z'])
    df['char'] = labels
    
    fig = px.scatter_3d(
        df, x='x', y='y', z='z', 
        text='char', 
        title='Mapa Mental 3D (Embeddings)',
        opacity=0.7
    )
    fig.update_traces(textposition='top center')
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=30))
    fig.show()

print("Función de visualización lista. Se llamará durante el entrenamiento.")

## ⚔️ Módulo 4: El Torneo Evolutivo

Aquí ocurre la magia. 
1.  Generamos una **Población** de modelos aleatorios.
2.  Los entrenamos brevemente (Sprint).
3.  Los peores mueren. Los mejores se reproducen (Crossover) y mutan.
4.  Repetimos.

In [ ]:
# Datos Dummy para probar el bucle (reemplazar con Scraper real)
dummy_data = torch.randint(0, 256, (1000,), dtype=torch.long)
def get_batch_dummy():
    ix = torch.randint(len(dummy_data) - 32, (16,))
    x = torch.stack([dummy_data[i:i+32] for i in ix]).to(device)
    y = torch.stack([dummy_data[i+1:i+33] for i in ix]).to(device)
    return x, y

def evolutionary_loop(generations=3, population_size=4):
    population = [Genome() for _ in range(population_size)]
    history = []
    
    print(f"🏁 Iniciando Evolución: {generations} Generaciones, {population_size} Individuos.")
    
    for gen in range(generations):
        print(f"\n--- Generación {gen+1} ---")
        
        # 1. Evaluar Fitness (Entrenar cada modelo)
        for i, genome in enumerate(population):
            print(f"Entrenando Individuo {i}: {genome.dna}")
            model = CortexOrganism(genome.dna).to(device)
            optimizer = torch.optim.AdamW(model.parameters(), lr=genome.dna['learning_rate'])
            
            # Mini-Entrenamiento (Sprint)
            losses = []
            model.train()
            for _ in range(20): # Pocas iteraciones para ser rápido
                xb, yb = get_batch_dummy()
                _, loss = model(xb, yb)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                losses.append(loss.item())
            
            # Fitness = Loss final (menor es mejor)
            genome.fitness = sum(losses[-5:]) / 5
            print(f"   -> Fitness (Loss): {genome.fitness:.4f}")
            
            # Visualizar el mejor de la generación
            if i == 0 and gen == generations - 1:
                visualize_brain_3d(model)

        # 2. Selección Natural (Torneo)
        population.sort(key=lambda x: x.fitness) # Ordenar por fitness (menor loss primero)
        survivors = population[:population_size//2] # Mueren el 50%
        print(f"💀 Han muerto {population_size - len(survivors)} individuos.")
        
        # 3. Reproducción y Mutación
        new_population = survivors[:]
        while len(new_population) < population_size:
            parent_a = random.choice(survivors)
            parent_b = random.choice(survivors)
            child = parent_a.crossover(parent_b)
            if random.random() < 0.3: # 30% Probabilidad de mutación
                print(f"   {child.mutate()}")
            new_population.append(child)
            
        population = new_population
        best_dna = survivors[0].dna
        history.append({'gen': gen, 'best_loss': survivors[0].fitness, 'dna': str(best_dna)})

    # Reporte Final
    print("\n🏆 Evolución Completada. El Modelo Perfecto es:")
    print(survivors[0].dna)
    
    # Gráfico de Evolución
    hist_df = pd.DataFrame(history)
    fig = px.line(hist_df, x='gen', y='best_loss', title='Evolución del Fitness (Loss)', markers=True)
    fig.show()

# ¡Ejecutar el Torneo!
evolutionary_loop()